## Generate geoJSON files for contours of inundation map

## References

1. [Extracting contours](https://stackoverflow.com/questions/18304722/python-find-contour-lines-from-matplotlib-pyplot-contour)
2. [Python Path objects](https://matplotlib.org/api/path_api.html)
3. [Geojson package](https://pypi.org/project/geojson/)
4. [Python Path tutorial](https://matplotlib.org/users/path_tutorial.html)
5. [Shapely package](https://pypi.org/project/Shapely/)

### First import some necessary tools
Note: you'll need to install the geojson module to run this notebook. If you have pip you can simply run
>pip install geojson

In [29]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [30]:
from __future__ import print_function
from ptha_paths import data_dir, events_dir
import sys, os
from matplotlib import _cntr as cntr
from geojson import MultiPolygon, Feature, FeatureCollection, dump
from skimage import measure
import geojson

### Set parameters 

In [31]:
# Parameters
events = ['AASZa', 'AASZb', 'AASZc', 'AASZd', 'CSZa', 'CSZb', 'CSZc', 'CSZd', 'CSZe', \
              'CSZf', 'KmSZa', 'KrSZa', 'SChSZa', 'TOHa']     # The events to consider
sch = ['SChSZa']
num_levels = 5      # The number of levels to partition inundation into
zeta_level_values = [1e-2] + list(linspace(0.5,4.5,num_levels-1))

In [32]:
zeta_level_values

[0.01, 0.5, 1.8333333333333333, 3.1666666666666665, 4.5]

### Read in the topography data and compute zeta-clines

In [33]:
# Read in topography data:
nx = 250
ny = 250

fixed_grid_file = os.path.join(data_dir, 'MapsTopo', 'fixedgrid_xyB_small.npy')
d=load(fixed_grid_file)
x=d[:,0] - 360   # Longitudes
y=d[:,1]         # Latitudes
B=d[:,2]         # Bathymetry (topography in the absence of water)
topo = reshape(B, (nx,ny), order='F')
X = reshape(x, (nx,ny), order='F')  # X[0,0] is min long, X[-1, 0] is max long
Y = reshape(y, (nx,ny), order='F')  # Y[0,0] is min lat,  Y[0, -1] is max lat

In [34]:
# Loop over all events and compute zeta clines for each
# Collect zeta clines for each level in multi-polygon environments
# for event in events:
for event in sch:
    event_dir = os.path.join(events_dir, event)
    hmax_file = os.path.join(event_dir, 'h_eta_small.npy')
    hmax = load(hmax_file)
    Hmax = hmax.reshape((nx,ny),order='F')
    
    features = [None] * num_levels
    c = cntr.Cntr(X,Y,Hmax)
    path_list = []
    for k in xrange(num_levels):
        # Go from largest values to smallest to get appropriate overlapping
        # contours = c.trace(zeta_level_values[num_levels-k-1])  # this returns paths and categorical variables. Separate..
        contours = c.trace(zeta_level_values[num_levels-k-1])  # this returns paths and categorical variables. Separate..
        num_paths = len(contours) // 2   # Half of the entries correspond to path connection types
        path_list = [poly.tolist() for poly in contours[:num_paths]]  # grab the first half
        multi_poly = MultiPolygon([path_list])
        features[num_levels-k-1] = Feature(geometry=multi_poly)
    # features = Feature(geometry=multi_poly)
    feature_collection = FeatureCollection(features)
    json_file = os.path.join(event_dir, 'contours.geojson')
    
    # Write to a file using geojson dump in the events dir (see first cell of this notebook)
    #with open(json_file, 'w') as outfile:
    #    dump(feature_collection, outfile)
    

# The next cell is an example of the Shapely package.

In [35]:
from shapely.geometry import polygon
for i in range(len(cs.collections)):
    p = cs.collections[i].get_paths()[0]
    v = p.vertices
    x = v[:,0]
    y = v[:,1]
    poly = Polygon([(j[0], j[1]) for j in zip(x,y)])
    print(i, poly)

0 Poly((-124.202, 41.7442) ...)
1 Poly((-124.202, 41.7442) ...)
2 Poly((-124.202, 41.7442) ...)
3 Poly((-124.202, 41.7398) ...)
4 Poly((-124.202, 41.7407) ...)
5 Poly((-124.202, 41.7433) ...)


IndexError: list index out of range

# This cell will generate the separate contour regions for the number of contour levels specified above. 

In [ ]:
# Shapely does not play well with the geoJSON package, so I will use some string methods from Python
# Based off https://gis.stackexchange.com/questions/99917/converting-matplotlib-contour-objects-to-shapely-objects
from shapely import geometry
import string
table = string.maketrans('()','[]')
table2 = string.maketrans('\'', '\"')
feature_collection = [None]*num_levels
for event in events:
    event_dir = os.path.join(events_dir, event)
    hmax_file = os.path.join(event_dir, 'h_eta_small.npy')
    hmax = load(hmax_file)
    Hmax = hmax.reshape((nx,ny),order='F')
    #mylevels = arange(0,16,.5);
    mylevels = zeta_level_values;
    #print(mylevels);
    cs = plt.contourf(X, Y, Hmax, mylevels)
    # First, loop through all zeta-levels
    feats = []
    #print(len(cs.collections))
    for kk in range(len(cs.collections)):
        col = cs.collections[kk];
        # feats = []  # feats is the payload to the "features" in geoJSON
        # Second, loop through all polygons that have the same zeta-level
        for contour_path in col.get_paths(): 
            # Create the polygons for this intensity level
            # The first polygon in iteration is the main polygon, the following ones are "holes"
            # Third, iterate through all sub-polygons that make up the main polygon
            for ncp,cp in enumerate(contour_path.to_polygons()):
                x = cp[:,0]
                y = cp[:,1]
                # new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                new_shape = geometry.LinearRing([(i[0], i[1]) for i in zip(x,y)])
                if ncp == 0:
                    first = [(i[0], i[1]) for i in zip(x,y)]
                    # poly = [new_shape]  # need to delete the holes in this inner-loop. This can be done by merging the polygons,
                    last = []
                else:
                    last.append([(i[0], i[1]) for i in zip(x,y)])
                    # poly = poly.append(new_shape)
                # instead of making a new polygon
            # print(poly[1])
            polyjson = {"geometry": geometry.mapping(geometry.Polygon(first, last)), "type": "Feature", "levelSetValue": mylevels[kk]}  # prep the payload
            feats.append(polyjson)
    json_file = os.path.join(event_dir, 'contours.geojson')

    # Write to a file in the events dir (see first cell of this notebook)
    with open(json_file, 'w') as outfile:
        outfile.write(str({"type": "FeatureCollection", "features": feats}).translate(table).translate(table2))
    # print(str({"type": "FeatureCollection", "features": feats}).translate(table).translate(table2))

#### Everything below is experiments with contour plots in matplotlib.

In [ ]:
from matplotlib import contour

In [ ]:
plt.contourf(X,Y,Hmax, zeta_level_values)

In [ ]:
## example see this link:
# https://stackoverflow.com/questions/18304722/python-find-contour-lines-from-matplotlib-pyplot-contour
xx = np.linspace(-1,1,nx)
XX = np.repeat(xx, nx).reshape(nx,ny)
YY = np.tile(xx, nx).reshape(nx,ny)
ZZ = XX**2 + YY**2
c = cntr.Cntr(XX, YY, ZZ)
contours = c.trace(0.5)  # the ZZ = 0.5 level-curve
num_paths = len(contours) // 2  # see this stackexchange link

In [ ]:
plt.contourf(XX,YY,ZZ,levels=[0,0.5,1,1.5,1.75])